<a href="https://colab.research.google.com/github/radve88/Learning-AI/blob/main/smolagent_RAG_with_smolagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Creating a RAG Tool for Guest Stories
Alfred, your trusted agent, is preparing for the most extravagant gala of the century. To ensure the event runs smoothly, Alfred needs quick access to up-to-date information about each guest. Let’s help Alfred by creating a custom Retrieval-Augmented Generation (RAG) tool, powered by our custom dataset.

Why RAG for a Gala?
Imagine Alfred mingling among the guests, needing to recall specific details about each person at a moment’s notice. A traditional LLM might struggle with this task because:

The guest list is specific to your event and not in the model’s training data
Guest information may change or be updated frequently
Alfred needs to retrieve precise details like email addresses
This is where Retrieval Augmented Generation (RAG) shines! By combining a retrieval system with an LLM, Alfred can access accurate, up-to-date information about your guests on demand.

You can choose any of the frameworks covered in the course for this use case. Select your preferred option from the code tabs.
Setting up our application
In this unit, we’ll develop our agent within a HF Space, as a structured Python project. This approach helps us maintain clean, modular code by organizing different functionalities into separate files. Also, this makes for a more realistic use case where you would deploy the application for public use.

Project Structure
tools.py – Provides auxiliary tools for the agent.
retriever.py – Implements retrieval functions to support knowledge access.
app.py – Integrates all components into a fully functional agent, which we’ll finalize in the last part of this unit.
For a hands-on reference, check out this HF Space, where the Agentic RAG developed in this unit is live. Feel free to clone it and experiment!
Dataset Overview
Our dataset agents-course/unit3-invitees contains the following fields for each guest:

Name: Guest’s full name
Relation: How the guest is related to the host
Description: A brief biography or interesting facts about the guest
Email Address: Contact information for sending invitations or follow-ups
Below is a preview of the dataset:

Step 1: Load and Prepare the Dataset

In [ ]:
import datasets
from langchain.docstore.document import Document

# Load the dataset
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

# Convert dataset entries into Document objects
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]


In the code above, we:

Load the dataset
Convert each guest entry into a Document object with formatted content
Store the Document objects in a list
This means we’ve got all of our data nicely available so we can get started with configuring our retrieval.
Step 2: Create the Retriever Tool
Now, let’s create a custom tool that Alfred can use to search through our guest information.
We will use the BM25Retriever from the langchain_community.retrievers module to create a retriever tool.

In [ ]:
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever

class GuestInfoRetrieverTool(Tool):
    name = "guest_info_retriever"
    description = "Retrieves detailed information about gala guests based on their name or relation."
    inputs = {
        "query": {
            "type": "string",
            "description": "The name or relation of the guest you want information about."
        }
    }
    output_type = "string"

    def __init__(self, docs):
        self.is_initialized = False
        self.retriever = BM25Retriever.from_documents(docs)

    def forward(self, query: str):
        results = self.retriever.get_relevant_documents(query)
        if results:
            return "\n\n".join([doc.page_content for doc in results[:3]])
        else:
            return "No matching guest information found."

# Initialize the tool
guest_info_tool = GuestInfoRetrieverTool(docs)

Let’s understand this tool step-by-step:

The name and description help the agent understand when and how to use this tool
The inputs define what parameters the tool expects (in this case, a search query)
We’re using a BM25Retriever, which is a powerful text retrieval algorithm that doesn’t require embeddings
The forward method processes the query and returns the most relevant guest information

Step 3: Integrate the Tool with Alfred
Finally, let’s bring everything together by creating our agent and equipping it with our custom tool:

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

# Initialize the Hugging Face model
model = InferenceClientModel()

# Create Alfred, our gala agent, with the guest info tool
alfred = CodeAgent(tools=[guest_info_tool], model=model)

# Example query Alfred might receive during the gala
response = alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("🎩 Alfred's Response:")
print(response)

What’s happening in this final step:

We initialize a Hugging Face model using the InferenceClientModel class
We create our agent (Alfred) as a CodeAgent, which can execute Python code to solve problems
We ask Alfred to retrieve information about a guest named “Lady Ada Lovelace”